In [5]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

Now it's time for another guided example. This time we're going to look at recipes. Specifically we'll use the epicurious dataset, which has a collection of recipes, key terms and ingredients, and their ratings.

What we want to see is if we can use the ingredient and keyword list to predict the rating. For someone writing a cookbook this could be really useful information that could help them choose which recipes to include because they're more likely to be enjoyed and therefore make the book more likely to be successful.

First let's load the dataset. It's [available on Kaggle](https://www.kaggle.com/hugodarwood/epirecipes). We'll use the csv file here and as pull out column names and some summary statistics for ratings.

In [6]:
raw_data = pd.read_csv('../../../Data & Script/epi_r.csv')

In [7]:
list(raw_data.columns)

['title',
 'rating',
 'calories',
 'protein',
 'fat',
 'sodium',
 '#cakeweek',
 '#wasteless',
 '22-minute meals',
 '3-ingredient recipes',
 '30 days of groceries',
 'advance prep required',
 'alabama',
 'alaska',
 'alcoholic',
 'almond',
 'amaretto',
 'anchovy',
 'anise',
 'anniversary',
 'anthony bourdain',
 'aperitif',
 'appetizer',
 'apple',
 'apple juice',
 'apricot',
 'arizona',
 'artichoke',
 'arugula',
 'asian pear',
 'asparagus',
 'aspen',
 'atlanta',
 'australia',
 'avocado',
 'back to school',
 'backyard bbq',
 'bacon',
 'bake',
 'banana',
 'barley',
 'basil',
 'bass',
 'bastille day',
 'bean',
 'beef',
 'beef rib',
 'beef shank',
 'beef tenderloin',
 'beer',
 'beet',
 'bell pepper',
 'berry',
 'beverly hills',
 'birthday',
 'biscuit',
 'bitters',
 'blackberry',
 'blender',
 'blue cheese',
 'blueberry',
 'boil',
 'bok choy',
 'bon appétit',
 'bon app��tit',
 'boston',
 'bourbon',
 'braise',
 'bran',
 'brandy',
 'bread',
 'breadcrumbs',
 'breakfast',
 'brie',
 'brine',
 'brisk

In [8]:
raw_data.rating.describe()

count    20052.000000
mean         3.714467
std          1.340829
min          0.000000
25%          3.750000
50%          4.375000
75%          4.375000
max          5.000000
Name: rating, dtype: float64

We learn a few things from this analysis. From a ratings perspective, there are just over 20,000 recipes with an average rating of 3.71. What is interesting is that the 25th percentile is actually above the mean. This means there is likely some kind of outlier population. This makes sense when we think about reviews: some bad recipes may have very few very low reviews.

Let's validate the idea a bit further with a histogram.

In [ ]:
raw_data.rating.hist(bins=20)
plt.title('Histogram of Recipe Ratings')
plt.show()

So a few things are shown in this histogram. Firstly there are sharp discontinutities. We don't have continuous data. No recipe has a 3.5 rating, for example. Also we see the anticipated increase at 0.

Let's try a naive approach again, this time using SVM Regressor. But first, we'll have to do a bit of data cleaning.

In [ ]:
# Count nulls 
null_count = raw_data.isnull().sum()
null_count[null_count > 0]

What we can see right away is that nutrition information is not available for all goods. Now this would be an interesting data point, but let's focus on ingredients and keywords right now. So we'll actually drop the whole columns for calories, protein, fat, and sodium. We'll come back to nutrition information later.

In [ ]:
# # takes long time
# from sklearn.svm import SVR
# svr = SVR()
X = raw_data.drop(['rating', 'title', 'calories', 'protein', 'fat', 'sodium'], 1)
y = raw_data.rating
# svr.fit(X,Y)

# load saved model
svr = pickle.load(open('svr1.sav', 'rb'))
svr

__Note that this actually takes quite a while to run, compared to some of the models we've done before. Be patient.__ It's because of the number of features we have.

Let's see what a scatter plot looks like, comparing actuals to predicted.

In [ ]:
plt.scatter(y, svr.predict(X))

Now that is a pretty useless visualization. This is because of the discontinous nature of our outcome variable. There's too much data for us to really see what's going on here. If you wanted to look at it you could create histograms, here we'll move on to the scores of both our full fit model and with cross validation. Again if you choose to run it again it will take some time, so you probably shouldn't.

In [ ]:
svr.score(X, Y)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(svr, X, Y, n_jobs=-1, cv=5)

Oh dear, so this did seem not to work very well. In fact it is remarkably poor. Now there are many things that we could do here. 

Firstly the overfit is a problem, even though it was poor in the first place. We could go back and clean up our feature set. There might be some gains to be made by getting rid of the noise.

We could also see how removing the nulls but including dietary information performs. Though its a slight change to the question we could still possibly get some improvements there.

Lastly, we could take our regression problem and turn it into a classifier. With this number of features and a discontinuous outcome, we might have better luck thinking of this as a classification problem. We could make it simpler still by instead of classifying on each possible value, group reviews to some decided high and low values.

__And that is your challenge.__

Transform this regression problem into a binary classifier and clean up the feature set. You can choose whether or not to include nutritional information, but try to cut your feature set down to the 30 most valuable features.

Good luck!

When you've finished that, also take a moment to think about bias. Is there anything in this dataset that makes you think it could be biased, perhaps extremely so?

There is. Several things in fact, but most glaringly is that we don't actually have a random sample. It could be, and probably is, that the people more likely to choose some kinds of recipes are more likely to give high reviews.

After all, people who eat chocolate _might_ just be happier people.

#### Attempt

* Recipe: title, rating (2 features)
* ingredients: calories, protein, fat, sodium (4 features)
* key_terms: (674 features)

Only key_terms are used as predictors. Rating is the target variable

Read this 
http://pbpython.com/categorical-encoding.html

### Multiclass Classification
#### Value Counts

In [ ]:
raw_data.rating.value_counts().sort_values()

In [ ]:
X = raw_data.drop(['rating', 'title', 'calories', 'protein', 'fat', 'sodium'], 1)
y = raw_data.rating
# convert y to categorical
from sklearn.preprocessing import LabelEncoder
lab_enc = LabelEncoder()
y = lab_enc.fit_transform(y)

In [ ]:
# after encoding
pd.Series(y).value_counts().sort_values()

The encoding somewhat arbitrary, It is ok for now. I can manually map them when needed.

#### Recursive Feature Elimination using Random Forest

In [ ]:
# Select Features using Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
# The "accuracy" scoring is proportional to the number of correct classifications
# takes 30 minutes, better to load previously saved model
selector_rf = pickle.load(open('selector_rf.sav', 'rb'))
#selector_rf = RFECV(RandomForestClassifier(), scoring='accuracy', n_jobs=-1)
#import time
#start_time = time.time()
#selector_rf.fit(X, y)
print("Optimal number of features : %d" % selector_rf.n_features_)
print(X.columns[selector_rf.get_support(indices=True)])
print(selector_rf.score(X,y))
#print("--- %s seconds ---" % (time.time() - start_time))
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(selector_rf.grid_scores_) + 1), selector_rf.grid_scores_)
plt.show()

In [ ]:
#trying to rank the features, but not working because the estimetor is not fitted using fit()
#selector_rf.estimator.feature_importances_

selector_rf.grid_scores_

#### PCA feature importance

In [ ]:
from sklearn.decomposition import PCA
sklearn_pca = PCA(n_components=30)
X_t = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)
# take the first principal component and find the loading scores(163 of them)
loading_scores = pd.Series(sklearn_pca.components_[0], index=X.columns)

sorted_loading_scores = loading_scores.abs().sort_values(ascending=False)

sorted_loading_scores.head(30)

In [ ]:
# the cofficents don't add up to 1, but the magnitude of an eigen vector is 1.0
print(np.linalg.norm(sklearn_pca.components_[0]))
print(np.linalg.norm(sklearn_pca.components_[1]))
print(np.linalg.norm(sklearn_pca.components_[2]))
print(np.linalg.norm(sklearn_pca.components_[3]))

In [ ]:
import time
start_time = time.time()
# train SVC model(takes time)
from sklearn.svm import SVC
#modelA =  SVC(tol=0.01, max_iter=-1)
# Use PCA features
#modelA.fit(X_t,y)
#print("--- %s seconds ---" % (time.time() - start_time))

modelA = pickle.load(open('modelA.sav', 'rb'))

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(modelA, X_t, y, n_jobs=-1, cv=5)

In [ ]:
# use original features
X = raw_data[sorted_loading_scores.index]
import time
start_time = time.time()
# train SVC model(takes time)
from sklearn.svm import SVC
#modelB =  SVC(tol=0.01, max_iter=-1)
#modelB.fit(X,y)
#print("--- %s seconds ---" % (time.time() - start_time))

modelB = pickle.load(open("modelB.sav", 'wb'))

In [ ]:
cross_val_score(modelB, X, y, n_jobs=-1, cv=5)

#### Feature Variance

In [ ]:
from sklearn.feature_selection import VarianceThreshold
# zero or one in 80% of the samples
# one class cannot be more than 91% of the data
var_selector = VarianceThreshold(.9115 * (1 - .9115))
X_t2 = var_selector.fit_transform(X)
print(X.columns[var_selector.get_support(indices=True)])

In [ ]:
# compare it with PCA
pca_and_variance_th = pd.DataFrame({"PCA_Importance": sorted_loading_scores.head(30).index, "Variance_Threshold":X.columns[var_selector.get_support(indices=True)]})
print(pca_and_variance_th)

# how much intersection
print(len(set(set(pca_and_variance_th.PCA_Importance).intersection(set(pca_and_variance_th.Variance_Threshold)))))

In [ ]:
import time
start_time = time.time()
# train SVC model(takes time)
from sklearn.svm import SVC
#modelC =  SVC(tol=0.01, max_iter=-1)
#modelC.fit(X_t2,y)
modelC = pickle.load(open('modelC.sav', 'rb'))
#print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
modelC.score(X_t2,y)

In [ ]:
# from sklearn.svm import SVC
# from sklearn.model_selection import StratifiedKFold
# from sklearn.feature_selection import RFECV 
# selector_three = RFECV(estimator= SVC(tol=0.01), step=1, cv=StratifiedKFold(10),
#               n_jobs=-1, scoring='accuracy')
# selector_three.fit(X, y)
# print(X.columns[selector_three.get_support(indices=True)])

# print("Optimal number of features : %d" % selector_three.n_features_)
# print(X.columns[selector_three.get_support(indices=True)])
# print("score: ", selector_three.score(X, y))
# # Plot number of features VS. cross-validation scores
# plt.figure()
# plt.xlabel("Number of features selected")
# plt.ylabel("Cross validation score (nb of correct classifications)")
# plt.plot(range(1, len(selector_three.grid_scores_) + 1), selector_three.grid_scores_)
# plt.show()

### Binary Classfier

In [ ]:
y = raw_data.rating
# split at the median
print(y.median())
print("Nice class balance: \n", np.bincount(np.where(y < 4.375, 0, 1)))
y_binary = np.where(y < 4.375, 0, 1)

In [ ]:
print("majority classifier output: ", 10738 / (10738 + 9314))

In [ ]:
## try PCA important features
# use original features
X = raw_data[sorted_loading_scores.index]
#import time
#start_time = time.time()
# train SVC model(takes time)
#from sklearn.svm import SVC
#modelD =  SVC(tol=0.01, max_iter=-1)
#modelD.fit(X, y_binary)
modelD = pickle.load(open('modelD.sav', 'rb'))
#print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
cross_val_score(modelD, X, y_binary, n_jobs=-1, cv=5)

In [ ]:
## try PCA components

#import time
#start_time = time.time()
# train SVC model(takes time)
from sklearn.svm import SVC
modelE = pickle.load(open('modelE.sav', 'rb'))

#modelE =  SVC(tol=0.01, max_iter=-1)
#modelE.fit(X_t, y_binary)
#print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
cross_val_score(modelE, X_t, y_binary, n_jobs=-1, cv=5)

In [ ]:
# Select Features using Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
# The "accuracy" scoring is proportional to the number of correct classifications
# takes 30 minutes, better to load previously saved model
selector_rf2 = pickle.load(open('selector_rf2.sav', 'rb'))
#selector_rf2 = RFECV(RandomForestClassifier(), scoring='accuracy', n_jobs=-1)
#import time
#start_time = time.time()
#selector_rf2.fit(X, y_binary)
print("Optimal number of features : %d" % selector_rf2.n_features_)
print(X.columns[selector_rf2.get_support(indices=True)])
print(selector_rf2.score(X, y_binary))
#print("--- %s seconds ---" % (time.time() - start_time))
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(selector_rf2.grid_scores_) + 1), selector_rf2.grid_scores_)
plt.show()

#### Recursive Feature Elimination(with Max features)

In [ ]:
import time
#start_time = time.time()
from sklearn.feature_selection import RFE
# takes 10 minutes
#selector_rf3 = RFE(RandomForestClassifier(), n_features_to_select=30, step=1)
#selector_rf3 = selector_rf3.fit(X, y_binary)
selector_rf3 = pickle.load(open("selector_rf3.sav","rb"))
#print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
print(X.columns[selector_rf3.get_support(indices=True)])

In [ ]:
# compare with the other feature selection results

allthree = pd.DataFrame({"PCA_Importance": sorted_loading_scores.head(30).index, 
                         "Variance_Threshold":X.columns[var_selector.get_support(indices=True)],
                         "Random Forest": X.columns[selector_rf3.get_support(indices=True)]
                        })
print(allthree)

In [ ]:
# transform X
Xt_rf = selector_rf3.transform(X)
Xt_rf.shape

In [ ]:
start_time = time.time()
#modelF =  SVC(tol=0.01, max_iter=-1)
#modelF.fit(X_t2,y_binary)
modelF = pickle.load(open('modelF.sav', 'rb'))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
cross_val_score(modelF, Xt_rf, y_binary, n_jobs=-1, cv=5)

In [ ]:
## Save the models
import pickle
pickle.dump(svr, open('svr1.sav', 'wb'))
pickle.dump(selector_rf, open('selector_rf.sav', 'wb'))
pickle.dump(modelA, open('modelA.sav', 'wb'))
pickle.dump(modelB, open('modelB.sav', 'wb'))
pickle.dump(modelC, open('modelC.sav', 'wb'))
pickle.dump(selector_rf2, open('selector_rf2.sav', 'wb'))
pickle.dump(modelD, open('modelD.sav', 'wb'))
pickle.dump(modelE, open('modelE.sav', 'wb'))
pickle.dump(selector_rf3, open('selector_rf3.sav', 'wb'))
pickle.dump(modelF, open('modelF.sav', 'wb'))

#### Reflection

Support Vector Machines can be used for Regression and Classification. For Classification we want to maximize the margin which is the distance from the classifier hyperplane(line for 2D data) to the support vectors(nereast vector to the boundary). If there are misclassified data points we want to minimize their distance to the boundary. For Regression, the margin is the difference between actual and predicted values. The advantage of SVR over other techniques like OLS is, we can specify the size of the margin and the penality for being outside the margin for training. 

There are many ways to select features. I run PCA and selected 30 features in two ways: the principal components(modelA), important features(modelB). Then I selected 30 features using VarianceThreshold(modelC). All performed similarly(around 0.4)
In this notebook, I tried to do multi-class SVC. 

For binary target variable, using the PCA fetures, the accuracy score is around 0.575. It is a little higher than the majority class(
I experimeted RFECV with randomforest for multiclass (rf1) and for binary ouput(rf2). Both yielded large number of features(>500). I couldn't not find a way to rank them. 

I found using RFE it is possible to get feature_importances. RFE with SVC is not possible because SVC don't have feature_importances or coef_ attributes. I used RFE with RandomForest and its accuracy is only 0.53.